<a href="https://colab.research.google.com/github/gollum-98/Crop-Disease-Identification---PlantVillage-Dataset/blob/master/Crop_Disease_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importing the necessary packages
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from PIL import Image
import os

In [2]:
#Cloning the PlantVillage-Dataset from Github, containing nd exciting dataset I found was the PlantVillage-Dataset by spMohanty on Github:  
#Containing raw, graysscale and segmented images of various categories of images of different plants – 
#both healthy and disease-ridden leaf images classified into 38 labels (54,306 images, 26 diseases, 14 crop species
!git clone https://github.com/spMohanty/PlantVillage-Dataset.git

Cloning into 'PlantVillage-Dataset'...
remote: Enumerating objects: 163229, done.
remote: Total 163229 (delta 0), reused 0 (delta 0), pack-reused 163229
Receiving objects: 100% (163229/163229), 2.00 GiB | 39.38 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Checking out files: 100% (182401/182401), done.


In [3]:
#Using Transfer Learning and using the VGG-19 architecture used for "Very Deep Convolutional Networks for Large-Scale Image Recognition"
pretrained_model = tf.keras.applications.VGG19(include_top=True,weights='imagenet',input_shape=(224,224,3))
#Keeping top layers included to summarize and view the top dense layers as well
pretrained_model.summary()

574717952/574710816 [==============================] - 4s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [4]:
pretrained_model = tf.keras.applications.VGG19(include_top=False,weights='imagenet',input_shape=(224,224,3))
#Using the VGG-19 model without the top dense layers
pretrained_model.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [5]:
#Freezing the layers of the VGG-19 model so that they are not trainable
for layer in pretrained_model.layers:
  layer.trainable = False
#pretrained_model.get_layer("block5_conv1").trainable = True
#pretrained_model.get_layer("block5_conv2").trainable = True
#pretrained_model.get_layer("block5_conv3").trainable = True
#pretrained_model.get_layer("block5_conv4").trainable = True
#pretrained_model.get_layer("block5_pool").trainable = True

In [11]:
#Getting output of last layer of VGG-19 model's frozen layers
last_layer = pretrained_model.get_layer("block5_pool")
output = last_layer.output
#Implementing the Top Dense Layers
X = tf.keras.layers.Flatten()(output)
X = tf.keras.layers.Dense(2048,activation="relu")(X)
X = tf.keras.layers.Dense(1024,activation="relu")(X)
X = tf.keras.layers.Dense(38,activation="softmax")(X)
model = tf.keras.models.Model(inputs=pretrained_model.input,outputs=X)
tf.keras.backend.clear_session()
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [12]:
#Preprocessing the data using Keras' ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1/255.0, rotation_range=20,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.2)
batch_size = 128
train_generator = train_datagen.flow_from_directory("/content/PlantVillage-Dataset/raw/color"
                                                    ,batch_size=128,target_size=(224,224),class_mode="categorical",subset="training")
validation_generator = train_datagen.flow_from_directory("/content/PlantVillage-Dataset/raw/color"
                                                    ,batch_size=128,target_size=(224,224),class_mode="categorical",subset="validation")
history = model.fit(train_generator,steps_per_epoch=train_generator.samples//batch_size,validation_data=validation_generator,validation_steps=validation_generator.samples//batch_size,epochs=20,verbose=1)

Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.
Epoch 1/20
339/339 [==============================] - 692s 2s/step - loss: 1.3115 - accuracy: 0.7257 - val_loss: 0.4660 - val_accuracy: 0.8503
Epoch 2/20
339/339 [==============================] - 681s 2s/step - loss: 0.3732 - accuracy: 0.8756 - val_loss: 0.3277 - val_accuracy: 0.8917
Epoch 3/20
339/339 [==============================] - 680s 2s/step - loss: 0.2993 - accuracy: 0.8990 - val_loss: 0.3283 - val_accuracy: 0.8895
Epoch 4/20
339/339 [==============================] - 681s 2s/step - loss: 0.2694 - accuracy: 0.9102 - val_loss: 0.2684 - val_accuracy: 0.9116
Epoch 5/20
339/339 [==============================] - 682s 2s/step - loss: 0.2332 - accuracy: 0.9209 - val_loss: 0.2947 - val_accuracy: 0.9074
Epoch 6/20
339/339 [==============================] - 682s 2s/step - loss: 0.2209 - accuracy: 0.9246 - val_loss: 0.2223 - val_accuracy: 0.9289
Epoch 7/20
339/339 [==============================] - 

In [ ]:
#Visualizing loss and accuracy over the training period for training and validation sets
loss = history.history['loss']
accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']
epoch = range(len(loss))
plt.figure()
plt.plot(epoch,loss)
plt.plot(epoch,val_loss)
plt.xlabel("Epoch#")
plt.ylabel("Accuracy/Loss")
plt.title("Loss")
plt.show()
plt.figure()
plt.plot(epoch,accuracy)
plt.plot(epoch,val_accuracy)
plt.xlabel("Epoch#")
plt.ylabel("Val Accuracy/Loss")
plt.title("Accuracy")
plt.show()